In [0]:
-- To list all catalogs
show catalogs;

-- To list all catalogs containing some keyword
show catalogs like '*e*'; -- Somehow different from PSQL where % is used as the wildcard character instead

In [0]:
-- To list all schemas in a catalog
show schemas in `dev`;

-- To list all schemas in a catalog containing some keyword
show schemas in `dev` like '*l*';

In [0]:
-- To list all tables in a schema
show tables in dev.silver;

-- To list all tables in a schema containing some keyword
show tables in dev.silver like '*ext*';

In [0]:
%python
# It is also possible to list all tables using the listTables() method.
spark.catalog.listTables('dev.bronze')

# And you can also check if a table exists using the tableExists() method.
spark.catalog.tableExists('dev.bronze.users') # False

In [0]:
-- To view the history of a delta table
describe history dev.bronze.data;

-- To query a particular instance in the history of a delta table
select * from dev.bronze.data@v1; -- v0 was the CREATE TABLE result

In [0]:
-- To view all kinds of metadata about a table
describe extended dev.bronze.data;

In [0]:
-- To create a temporary view that is lost when the session is destroyed
create or replace temporary view data_temp as select * from dev.bronze.data where data_id in (1, 2, 3); -- Does not take in catalog.schema.view naming format, only view name

select * from data_temp;

In [0]:
-- To create a view that persists
create or replace view dev.bronze.data_temp as select * from dev.bronze.data where data_id in (2, 3);

select * from dev.bronze.data_temp;

In [0]:
-- CTAS - Create table as select
-- Duplicates to a different location
create or replace table dev.bronze.data_ctas as
select * from dev.bronze.data;

In [0]:
describe extended dev.bronze.data;

In [0]:
-- Deep clone
-- Copies both data and metadata
create or replace table dev.bronze.data_deep_clone
deep clone dev.bronze.data;

describe extended dev.bronze.data_deep_clone; -- Exact metadata and all properties

In [0]:
-- Shallow clone
-- Copies metadata only; data points to location where original data is stored
create or replace table dev.bronze.data_shallow_clone
shallow clone dev.bronze.data;

describe extended dev.bronze.data_shallow_clone;
-- Only the metadata is copied; data points to the original location
-- If new data is added to the original table, it is not reflected in the shallow clone because it refers to a particular version in the history of the original table
-- If new data is added to the clone, the original table is not affected; the new data record is stored in the clone table's location

In [0]:
-- Create dummy pre and post tables
create or replace table dev.bronze.data_pre
(
  data_id int,
  value string
);

insert into dev.bronze.data_pre values (1, "ABC");
insert into dev.bronze.data_pre values (2, "DEF");
insert into dev.bronze.data_pre values (3, "GHI");

create or replace table dev.bronze.data_post
(
  data_id int,
  value string
);

insert into dev.bronze.data_post values (1, "ABC-new");
insert into dev.bronze.data_post values (2, "DEF-new");
insert into dev.bronze.data_post values (4, "JKL");

In [0]:
select * from dev.bronze.data_pre;

In [0]:
select * from dev.bronze.data_post;

In [0]:
-- Create dummy final table in silver
create or replace table dev.silver.data_final as
select * from dev.bronze.data_pre;

In [0]:
select * from dev.silver.data_final;

In [0]:
-- Merge - upsert
merge into dev.silver.data_final m
using dev.bronze.data_post u
on m.data_id = u.data_id
when matched then
update set m.value = u.value -- update if match
when not matched then
insert *; -- insert if id does not exist

In [0]:
select * from dev.silver.data_final; -- updated data_id = 1 and 2, inserted data_id = 4

In [0]:
-- Merge - delete from target if not found
merge into dev.silver.data_final m
using dev.bronze.data_post u
on m.data_id = u.data_id
when matched then
update set m.value = u.value -- update if match
when not matched then
insert * -- insert if id does not exist
when not matched by source then
delete; -- delete if id does not exist in source

In [0]:
select * from dev.silver.data_final; -- deleted data_id = 3 from target because key was not in source

In [0]:
-- Merge - soft delete from target if not found (add some kind of flag column)
-- First alter table to add flag column
alter table dev.silver.data_final add column status string;

-- Merge conditions
merge into dev.silver.data_final m
using dev.bronze.data_post u
on m.data_id = u.data_id
when matched then
update set m.value = u.value, m.status = 'updated' -- update if match
when not matched then
insert 
(
  data_id,
  value,
  status
)
values
(
  u.data_id,
  u.value,
  'new'
) -- insert if id does not exist, columns and values need to be specified here since there is an extra column added here
when not matched by source then
update set status = 'inactive'; -- update status to 'inactive' if id does not exist in source

In [0]:
select * from dev.silver.data_final;

In [0]:
update dev.silver.data_final set status = 'active' where status = 'null' or status is null;

In [0]:
-- Deletion vectors
-- Often datasets come partitioned in multiple parquet or csv files or such. Deleting a few records from a Delta Live Table may require the deletion and writing of multiple files in the Delta Lake. This would be inefficient. To avoid this, Delta Live Tables has a feature called Deleteion Vectors which are flags that specify a file or files corresponding to a record needs to be deleted during maintenance updates of the tables or the Delta Lake when the record itself is deleted. This allows delete operations to be sort of batched together in the Delta Lake and prevents excessive file reads/writes. One can also make the file modifications instantly by optimizing the table using the OPTIMIZE command.
-- Deletion vectors are enabled by default.
create or replace table dev.bronze.retail_data_new as
select * from
read_files( -- This is a utility to read CSV files from a directory
  'dbfs:/databricks-datasets/online_retail/data-001/data.csv',
  format => 'csv',
  header => true
);

In [0]:
select * from dev.bronze.retail_data_new;

In [0]:
describe extended dev.bronze.retail_data_new;
-- In Table Properties, it says delta.enableDeletionVectors=True

In [0]:
-- Disable deletion vectors and observe deletion operation effects
alter table dev.bronze.retail_data_new set tblproperties (delta.enableDeletionVectors = False);

delete from dev.bronze.retail_data_new where InvoiceNo = 540644;

describe history dev.bronze.retail_data_new;
-- We deleted 35 rows, 2 files and added 1 file per operationMetrics from history

In [0]:
-- Re-enable deletion vectors and observe deletion operation effects
alter table dev.bronze.retail_data_new set tblproperties (delta.enableDeletionVectors = True);

delete from dev.bronze.retail_data_new where InvoiceNo = 536368;

describe history dev.bronze.retail_data_new;
-- We added 1 deletion vector to delete 4 rows, removed and wrote 0 files per operationMetrics from history

In [0]:
-- Liquid clustering
-- Liquid clustering is a way to manage the size of files behind a table when the number of
-- partitions is too little or too high. It is possible to achieve a similar effect with
-- ZORDER and OPTIMIZE, but liquid cluetering is better since there is no need to rewrite
-- the entire data for incremental data updates. Liquid clustering also takes care of
-- repartitioning the file sizes for the selected clusters. These optimizations make
-- queries and operations much faster.
-- (??)

-- Set cluster columns
alter table dev.bronze.retail_data_new cluster by (InvoiceNo); -- This updates a few protocols on the tables

-- Also possible to assign clustering on CT or CTAS command
-- create or replace table dev.bronze.retail_data_new CLUSTER BY (InvoiceNo) as
-- select * from
-- read_files( -- This is a utility to read CSV files from a directory
--   'dbfs:/databricks-datasets/online_retail/data-001/data.csv',
--   format => 'csv',
--   header => true
-- );

In [0]:
-- Querying (having the clustering column in filter or predicate) by clustering column
select * from dev.bronze.retail_data_new where InvoiceNo = 536367;